In [1]:
import warnings
warnings.simplefilter('ignore')

import dill
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Import the csv Data as Pnadas dataframe
df = pd.read_csv("data/apple_quality.csv")
df.sample(5)

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
3443,3443.0,-0.311886,-0.311816,-0.476390,-0.216676,0.784805,0.980858,-1.812256803,good
1403,1403.0,-0.233342,3.440811,-1.195862,1.081771,3.178556,-2.824407,2.012270168,good
2128,2128.0,-0.792714,-0.229621,-3.055723,1.945335,-0.509488,2.215652,0.1481226,bad
1794,1794.0,-0.852608,-2.409377,-4.204690,0.284769,-2.557982,4.092179,-2.878008873,bad
2184,2184.0,-0.129502,-4.690071,1.126250,3.036022,2.295715,2.492054,-0.476322949,good


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4001 entries, 0 to 4000
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   float64
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4001 non-null   object 
 8   Quality      4000 non-null   object 
dtypes: float64(7), object(2)
memory usage: 281.4+ KB


In [4]:
df.drop(columns="A_id", inplace=True)

In [5]:
df.dropna(inplace=True)

In [6]:
df['Acidity']=df['Acidity'].astype(float)

In [7]:
# Prepare the X and Y variables
x = df.drop(columns=['Quality'])
y=df['Quality']
print('x shape : ', x.shape)
print('y shape : ',y.shape)

x shape :  (4000, 7)
y shape :  (4000,)


In [8]:
# Create group of the Numerical Features and Categorical Feature 
num_features = x.select_dtypes(exclude="O").columns
cat_features = x.select_dtypes(include="O").columns

In [9]:
print('*'*50)
print('Numeric Features : ', num_features)
print('*'*50)
print('Categorical Features : ', cat_features)
print('*'*50)

**************************************************
Numeric Features :  Index(['Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Ripeness',
       'Acidity'],
      dtype='object')
**************************************************
Categorical Features :  Index([], dtype='object')
**************************************************


In [10]:
from sklearn.impute import SimpleImputer  # handling missing values
from sklearn.preprocessing import StandardScaler  # handling featuer scaling
from sklearn.preprocessing import OneHotEncoder, LabelEncoder   # ordinal encoding
from sklearn.pipeline import Pipeline    # pipeline
from sklearn.compose import ColumnTransformer
le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
for i in cat_features:
    print(f"Categories in '{i} variable : ", end=" ")
    print(df[i].unique())
    print("======================================================")

In [12]:
# Numerical Pipeline

num_pipeline = Pipeline(
    steps=[
        ('SimpleImputer',SimpleImputer(strategy='median')),
        ('StandardScaler',StandardScaler())
    ]
)

# Categorical Pipeline

cat_pipeline = Pipeline(
    steps=[
        ('SimpleImputer',SimpleImputer(strategy='most_frequent')),
        ('oneHot',OneHotEncoder(sparse=False, drop='first', dtype=np.int16))
    ]
)

# combine num_pipeline and cat_pipeline

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,num_features),
    ('categorical_pipeline',cat_pipeline,cat_features)
])

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3200, 7), (800, 7), (3200,), (800,))

In [17]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [18]:
# Create the Evaluate Model
def evaluate_model(true, predicted):
    print("accuracy_score")
    accuracy_score1 = accuracy_score(true, predicted)
    print("classification_report")
    classification_report1 = classification_report(true, predicted)
    print("confusion_matrix")
    confusion_matrix1 = confusion_matrix(true, predicted)
    return accuracy_score1, classification_report1, confusion_matrix1

In [21]:
models = {
    "LogisticRegression":LogisticRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "KNeighborsClassifier":KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(),
    "Random Forest Classifier":RandomForestClassifier(),
    "XGBClassifier": XGBClassifier(n_jobs=-1),
    "CatBoostClassifier":CatBoostClassifier(verbose=True),
    "SVC": SVC(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "GaussianNB": GaussianNB(),
}

In [22]:
model_list = []
acuracy_score_list = []
class_report = []
matrics = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    model_accuracy, model_classification_report, model_confusion_matrix = evaluate_model(y_test, y_pred.round())
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Performance for Testing set")
    print("Model accuarcy : {}".format(model_accuracy))
    print("="*50)
    print("classification_report :")
    print(model_classification_report)
    print("="*50)
    print("Confusion Matrix")
    print(model_confusion_matrix)

    acuracy_score_list.append(model_accuracy)
    class_report.append(model_classification_report)
    matrics.append(model_confusion_matrix)
    print("*"*50)
    print()

accuracy_score
classification_report
confusion_matrix
LogisticRegression
Model Performance for Testing set
Model accuarcy : 0.75375
classification_report :
              precision    recall  f1-score   support

           0       0.75      0.76      0.75       401
           1       0.75      0.75      0.75       399

    accuracy                           0.75       800
   macro avg       0.75      0.75      0.75       800
weighted avg       0.75      0.75      0.75       800

Confusion Matrix
[[303  98]
 [ 99 300]]
**************************************************

accuracy_score
classification_report
confusion_matrix
Lasso
Model Performance for Testing set
Model accuarcy : 0.49875
classification_report :
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       401
           1       0.50      1.00      0.67       399

    accuracy                           0.50       800
   macro avg       0.25      0.50      0.33       800
weighted a